In [ ]:
!pip install pycryptodome

In [88]:
import numpy as np
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import base64
import random

In [215]:
# RSA Operations
def generate_rsa_keys():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return public_key, private_key

def rsa_encrypt(public_key, message):
    public_key = RSA.import_key(public_key)
    cipher = PKCS1_OAEP.new(public_key)
    encrypted_message = cipher.encrypt(message)
    return base64.b64encode(encrypted_message)

def rsa_decrypt(private_key, encrypted_message):
    private_key = RSA.import_key(private_key)
    cipher = PKCS1_OAEP.new(private_key)
    decrypted_message = cipher.decrypt(base64.b64decode(encrypted_message))
    return decrypted_message

In [216]:
def gram_schmidt(basis):
    orthogonal_basis = np.zeros(basis.shape)
    for i in range(len(basis)):
        orthogonal_basis[i] = basis[i]
        for j in range(i):
            denominator = np.dot(orthogonal_basis[j], orthogonal_basis[j])
            if denominator != 0:
                orthogonal_basis[i] -= np.dot(basis[i], orthogonal_basis[j]) / denominator * orthogonal_basis[j]
    return orthogonal_basis

def lll_reduction(basis, delta=0.75):
    reduced_basis = np.array(basis, dtype=float)
    orthogonal_basis = gram_schmidt(reduced_basis)
    k = 1
    while k < len(basis):
        for j in range(k-1, -1, -1):
            denominator = np.dot(orthogonal_basis[j], orthogonal_basis[j])
            if denominator != 0:
                mu = np.dot(reduced_basis[k], orthogonal_basis[j]) / denominator
                if abs(mu) > 0.5:
                    reduced_basis[k] -= mu * reduced_basis[j]
                    orthogonal_basis = gram_schmidt(reduced_basis)

        denominator = np.dot(orthogonal_basis[k-1], orthogonal_basis[k-1])
        if denominator != 0:
            mu_k = np.dot(reduced_basis[k], orthogonal_basis[k-1]) / denominator
            if np.linalg.norm(orthogonal_basis[k]) ** 2 >= (delta - mu_k ** 2) * np.linalg.norm(orthogonal_basis[k-1]) ** 2:
                k += 1
            else:
                reduced_basis[[k, k-1]] = reduced_basis[[k-1, k]]
                orthogonal_basis = gram_schmidt(reduced_basis)
                k = max(k-1, 1)

    return reduced_basis

In [217]:
# Lattice-Based Operations
def create_lattice():
    rows, cols = random.randint(3, 3), random.randint(3, 3)
    lattice = np.random.randint(0, 1000, (rows, cols))
    return lattice

def solve_lattice_problem(lattice):
    reduced_basis = lll_reduction(lattice)
    check_lovasz_condition(reduced_basis)
    reduced_basis = sorted(reduced_basis, key=lambda v: np.linalg.norm(v))
    shortest_vector = reduced_basis[0]
    return shortest_vector

def check_lovasz_condition(reduced_basis, delta=0.75):
    """Check if the Lovász condition is satisfied for the reduced basis."""
    orthogonal_basis = gram_schmidt(reduced_basis)
    for i in range(len(reduced_basis) - 1):
        left_side = np.linalg.norm(orthogonal_basis[i + 1])**2
        projection = np.dot(reduced_basis[i + 1], orthogonal_basis[i]) / np.dot(orthogonal_basis[i], orthogonal_basis[i])
        right_side = (delta - projection**2) * np.linalg.norm(orthogonal_basis[i])**2
        if left_side < right_side:
            print(f"Lovász condition NOT satisfied between vectors {i + 1} and {i + 2}.")
            return False
    print("Lovász condition satisfied for all consecutive vector pairs.\n")
    return True

def lattice_encrypt(lattice, key):
    shortest_vector = solve_lattice_problem(lattice)
    encrypted_data = shortest_vector + key
    return encrypted_data

def lattice_decrypt(lattice, encrypted_data):
    shortest_vector = solve_lattice_problem(lattice)
    decrypted_data = encrypted_data - shortest_vector
    return decrypted_data

In [219]:
public_key, private_key = generate_rsa_keys()
lattice_key = np.random.randint(1, 10, size=(3,))  # Random key for lattice encryption
encrypted_key = rsa_encrypt(public_key, lattice_key.tobytes())  # Encrypting lattice key with RSA
decrypted_key = rsa_decrypt(private_key, encrypted_key)  # Decrypting with RSA

# Convert bytes back to numpy array
lattice_key_decrypted = np.frombuffer(decrypted_key, dtype=int)

# Lattice operations
lattice = create_lattice()
encrypted_data = lattice_encrypt(lattice, lattice_key_decrypted)
decrypted_data = lattice_decrypt(lattice, encrypted_data)

print("Lattice Key (Original):", lattice_key)
print("Lattice Key (Decrypted):", lattice_key_decrypted)
print("Encrypted Data:", encrypted_data)
print("Decrypted Data:", decrypted_data)

Lovász condition satisfied for all consecutive vector pairs.

Lovász condition satisfied for all consecutive vector pairs.

Lattice Key (Original): [8 8 7]
Lattice Key (Decrypted): [8 8 7]
Encrypted Data: [ 83.84629853  21.64631802 -71.7463291 ]
Decrypted Data: [8. 8. 7.]
